In [10]:
import sklearn
import os 
import pandas as pd
import numpy as np

**IMPORTING FILES**

In [11]:
#reads files and creates list
def filereader(path):
    list_of_comments = []
    all_files = os.listdir(path)
    for folder in all_files:
        files = os.listdir(path+folder)
        for file in files:
            pathname = path+folder+'\\'+file
            file = open(pathname, 'r')
            list_of_comments.append(file.readline())
    return list_of_comments

In [12]:
neg_pol_truthful = filereader("C:\\Users\\aligthar\\Desktop\\negative_polarity\\truthful_from_Web\\")
neg_pol_deceptive = filereader("C:\\Users\\aligthar\\Desktop\\negative_polarity\\deceptive_from_MTurk\\")
pos_pol_truthful = filereader("C:\\Users\\aligthar\\Desktop\\positive_polarity\\truthful_from_TripAdvisor\\")
pos_pol_deceptive = filereader("C:\\Users\\aligthar\\Desktop\\positive_polarity\\deceptive_from_MTurk\\")

**PREPROCESSING**

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from random import shuffle

In [14]:
def preprocessing(text, pos_tags=False):
    # tokenize into words
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]   

    # remove stopwords
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]

    # remove words less than three characters
    tokens = [word for word in tokens if len(word) >= 3]

    # remove capitalization
    tokens = [word.lower() for word in tokens]

    # lemmatizing
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]

    # includes POS_tags
    if pos_tags:
        tokens = nltk.pos_tag(tokens)
        tokens = ['_'.join(t) for t in tokens]
    
    preprocessed_text= ' '.join(tokens)

    return preprocessed_text 

def preprocesser(list_of_texts): # turning lists into preprocessed lists
    new = []
    for item in list_of_texts:
        new += [preprocessing(item)]
    return new

In [15]:
# labelling data: 0 for truthful, 1 for deceptive
corpus_neg_truthful = [(r,0) for r in preprocesser(neg_pol_truthful)]
corpus_neg_deceptive = [(r,1) for r in preprocesser(neg_pol_deceptive)]
corpus_pos_truthful = [(r,0) for r in preprocesser(pos_pol_truthful)]
corpus_pos_deceptive = [(r,1) for r in preprocesser(pos_pol_deceptive)]

In [16]:
# shuffle data instances
from random import shuffle
corpus = corpus_neg_truthful+corpus_neg_deceptive+corpus_pos_truthful+corpus_pos_deceptive
shuffle(corpus)
reviews = [r for r,l in corpus]
labels_y = [l for r,l in corpus]

**SUPERVISED LEARNING**

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from self_learning.frameworks.SelfLearning import *
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from statistics import mean, stdev
from datetime import datetime

In [18]:
def main_CV(ratio, vectorizertype, ngram, nsplits):
    
    """
    Main function with cross-validation included
    Parameters:
        ratio (number of labels to remove) = float (0.80 / 0.90 / 0.95)
        vectorizertype (word counts or TF-IDF values) = CountVectorizer / TfidfVectorizer
        ngram (word pairs) = int (1=unigram, 2=bigram)
        nsplits (cross validation splits) = int (number of splits)
    
    """  
    SVM_results = {'accuracy':[], '1':{'precision':[], 'recall':[], 'f1-score':[]}}

    NB_results = {'accuracy':[], '1':{'precision':[], 'recall':[], 'f1-score':[]}}
    
    RF_results = {'accuracy':[], '1':{'precision':[], 'recall':[], 'f1-score':[]}}
    
    # creating dataframe 
    vectorizer = vectorizertype(min_df=2, max_df=0.5, ngram_range=(1,ngram))
    X = vectorizer.fit_transform(reviews)
    DF = pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names())
    DF['Y'] = labels_y
    X = DF.drop('Y', axis=1)
    y = DF['Y']
    
    # feature selection
    def chi_square(X, y): 
        F, pval = chi2(X, y) 
        return F
    def feature_ranking(F):
        idx = np.argsort(F)
        return idx[::-1]
    df = pd.DataFrame(feature_ranking(chi_square(X,y)))
    df.columns = ['column_numbers']
    clms_toselect = df['column_numbers'].head(1000)
    a = [DF.columns[i] for i in clms_toselect]
    X = DF[a]
    X = X.to_numpy()
    
    # removing data instances
    y = y.to_numpy()
    X = X[0:int((1-ratio)*len(X))] 
    y = y[0:int((1-ratio)*len(y))]
    
    # initialize classifiers 
    SVM = SVC(probability=True, kernel='rbf', gamma='scale', C=9)
    NB = MultinomialNB(alpha=0.7)
    RF = RandomForestClassifier(n_estimators=150, max_depth=30, min_samples_split=20)
    
    # KFold crossvalidation
    kf = KFold(n_splits=nsplits, shuffle=False)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # train models
        SVM.fit(X_train, y_train)
        NB.fit(X_train, y_train)
        RF.fit(X_train, y_train)

        # evaluate models 
        y_pred_SVM = SVM.predict(X_test)
        y_pred_NB = NB.predict(X_test)
        y_pred_RF = RF.predict(X_test)
    
        report = classification_report(y_test,y_pred_SVM, output_dict=True)
        SVM_results['accuracy'] += [report['accuracy']]
        SVM_results['1']['precision'] += [report['1']['precision']]
        SVM_results['1']['recall'] += [report['1']['recall']]
        SVM_results['1']['f1-score'] += [report['1']['f1-score']]
          
        report = classification_report(y_test,y_pred_NB, output_dict=True)
        NB_results['accuracy'] += [report['accuracy']]
        NB_results['1']['precision'] += [report['1']['precision']]
        NB_results['1']['recall'] += [report['1']['recall']]
        NB_results['1']['f1-score'] += [report['1']['f1-score']]
     
        report = classification_report(y_test,y_pred_RF, output_dict=True)
        RF_results['accuracy'] += [report['accuracy']]
        RF_results['1']['precision'] += [report['1']['precision']]
        RF_results['1']['recall'] += [report['1']['recall']]
        RF_results['1']['f1-score'] += [report['1']['f1-score']]
        
    
    with open('Supervised_Classifiers_results.txt', 'a') as file:
        file.write('___ratio:' + str(ratio) + '|ngram:' + str(ngram) + '|vectorizer:'+ str(vectorizertype) + '___\n')
        file.write('SVM:\n')
        file.write('accuracy: ' + str(round(mean(SVM_results['accuracy']),2)) + '\n')
        file.write('stddev: ' + str(stdev(SVM_results['accuracy'])) + '\n')
        file.write('precision: ' + str(round(mean(SVM_results['1']['precision']),2)) + '\n')
        file.write('recall: ' + str(round(mean(SVM_results['1']['recall']),2)) + '\n')
        file.write('F1 :' + str(round(mean(SVM_results['1']['f1-score']),2)) + '\n\n')

        file.write('NB:\n')        
        file.write('accuracy: ' + str(round(mean(NB_results['accuracy']),2)) + '\n')
        file.write('stddev: ' + str(stdev(NB_results['accuracy'])) + '\n')
        file.write('precision: ' + str(round(mean(NB_results['1']['precision']),2)) + '\n')
        file.write('recall: ' + str(round(mean(NB_results['1']['recall']),2)) + '\n')
        file.write('F1 :' + str(round(mean(NB_results['1']['f1-score']),2)) + '\n\n')
        
        file.write('RF:\n')        
        file.write('accuracy: ' + str(round(mean(RF_results['accuracy']),2))+ '\n')
        file.write('stddev: ' + str(stdev(RF_results['accuracy'])) + '\n')
        file.write('precision: ' + str(round(mean(RF_results['1']['precision']),2)) + '\n')
        file.write('recall: ' + str(round(mean(RF_results['1']['recall']),2)) + '\n')
        file.write('F1 :' + str(round(mean(RF_results['1']['f1-score']),2)) + '\n\n')
        obj = datetime.now()
        file.write(str(obj) + '\n\n\n')

In [19]:
main_CV(ratio=0.80, vectorizertype=CountVectorizer, ngram=1, nsplits=5)